In [0]:
%pip install groq


In [0]:
dbutils.library.restartPython() 

In [0]:
from groq import Groq

client = Groq(api_key="your_api_key")

def gen_marketing_campaign(district, city, ingredient):
    prompt = f"""
Create a seasonal cookie recipe description for {district}, {city}
using {ingredient}.

Format:
1. Local essence
2. Lifestyle appeal
3. Positive calorie classification
"""

    completion = client.chat.completions.create(
        model="llama-3.1-8b-instant",
        messages=[{"role": "user", "content": prompt}],
        temperature=0.7
    )

    return completion.choices[0].message.content


In [0]:
top_df = spark.table("daiwt_gourmet.gourmet_data.gold_flagship_products")

In [0]:
rows = top_df.collect()

results = []

for r in rows:
    description = gen_marketing_campaign(
        r.franchise_district,
        r.franchise_city,
        r.supplier_ingredient
    )
    
    results.append((
        *r,
        description
    ))


In [0]:
columns = top_df.columns + ["description"]

flagship_stores_df = spark.createDataFrame(results, columns)


In [0]:
flagship_stores_df.write.mode("overwrite").saveAsTable("daiwt_gourmet.gourmet_data.flagship_stores")


In [0]:
df=spark.read.table("daiwt_gourmet.gourmet_data.flagship_stores")
display(df)